# Notes on Scripts

## FEM files

``` python
Core FEM files are

* Output_Integrator.csv: 2752 rows (I think these are the integrals of the nodes wrt the elemnt surface ??)
gradNa_node1_x,gradNa_node1_y,gradNa_node2_x,gradNa_node2_y,gradNa_node3_x,gradNa_node3_y,qpWeight

* output_elements.csv: 2752 rows
node1,node2,node3
144,441,443
....

* output_nodes.csv: 1441 rows
x,y,ux,uy,fintx,finty,bcx,bcy,ux_orig,uy_orig

* output_reactions.csv: 
forces
-1.78184371484038400757e-01
1.78184350922636763892e-01
-1.99714523760122358853e-01
1.99713058872547727374e-01


## Drivers

### Train.py

```python
# Forward pass NN to obtain W_NN
W_NN = model(torch.cat((F11,F12,F21,F22),dim=1))
```

--------------------------------------------------

When printing the shape of this, it is: 
```python

Model output shape 
torch.Size([2752, 1])

Model input shape 
torch.Size([2752, 4])
```

So it prints I think, the energy density per node?

Vars they work with:
```python
dict_keys(['path', 'numNodes', 'x_nodes', 'u_nodes', 'dirichlet_nodes', 'reactions', 'numElements', 'qpWeights', 'connectivity', 'gradNa', 'F', 'J', 'C', 'I1', 'I2', 'I3', 'dI1dF', 'dI2dF', 'dI3dF', 'featureSet'])

```python

OK by printing: print(data.numElements)
we get that there are 2752 elements.
By printing numNodes we find 1441 nodes.
Printing dirichletNodes shape we find torch.Size([1441, 2])
Printing x_nodes shape we find torch.Size([1441, 2])
Printing u_nodes shape we find torch.Size([1441, 2])

In [2]:
2752/1441

1.9097848716169328

Calculate first Piola–Kirchhoff stress:

``` python

# Get gradients of W w.r.t F
dW_NN_dF11 = torch.autograd.grad(W_NN,F11,torch.ones(F11.shape[0],1),create_graph=True)[0]
dW_NN_dF12 = torch.autograd.grad(W_NN,F12,torch.ones(F12.shape[0],1),create_graph=True)[0]
dW_NN_dF21 = torch.autograd.grad(W_NN,F21,torch.ones(F21.shape[0],1),create_graph=True)[0]
dW_NN_dF22 = torch.autograd.grad(W_NN,F22,torch.ones(F22.shape[0],1),create_graph=True)[0]

# Assemble First Piola-Kirchhoff stress components
P_NN = torch.cat((dW_NN_dF11,dW_NN_dF12,dW_NN_dF21,dW_NN_dF22),dim=1)
``` 

Correction is so that PK stress is 0 at reference state

``` python

# Forward pass to obtain zero deformation energy correction
W_NN_0 = model(torch.cat((F11_0,F12_0,F21_0,F22_0),dim=1))

# Get gradients of W_NN_0 w.r.t F
dW_NN_dF11_0 = torch.autograd.grad(W_NN_0,F11_0,torch.ones(F11_0.shape[0],1),create_graph=True)[0]
dW_NN_dF12_0 = torch.autograd.grad(W_NN_0,F12_0,torch.ones(F12_0.shape[0],1),create_graph=True)[0]
dW_NN_dF21_0 = torch.autograd.grad(W_NN_0,F21_0,torch.ones(F21_0.shape[0],1),create_graph=True)[0]
dW_NN_dF22_0 = torch.autograd.grad(W_NN_0,F22_0,torch.ones(F22_0.shape[0],1),create_graph=True)[0]

# Get stress at zero deformation
P_NN_0 = torch.cat((dW_NN_dF11_0,dW_NN_dF12_0,dW_NN_dF21_0,dW_NN_dF22_0),dim=1)

# Initialize zero stress correction term
P_cor = torch.zeros_like(P_NN)

# Compute stress correction components according to Ansatz
P_cor[:,0:1] = F11*-P_NN_0[:,0:1] + F12*-P_NN_0[:,2:3]
P_cor[:,1:2] = F11*-P_NN_0[:,1:2] + F12*-P_NN_0[:,3:4]
P_cor[:,2:3] = F21*-P_NN_0[:,0:1] + F22*-P_NN_0[:,2:3]
P_cor[:,3:4] = F21*-P_NN_0[:,1:2] + F22*-P_NN_0[:,3:4]

# Compute final stress (NN + correction)
P = P_NN + P_cor


## FEM data

## Core